# Lab 04

In this lab, we will first create a dataset for Linear regression, then build different methods to solve linear regression in 3D.

We will go from generating synthetic data to optimizing the parameters of our model and plotting the results.

## Exercise 1

Build a class to generate 3D lines given line parameters, and noise levels:
- the noise Eps is Normal centered on 0, with a parameterized variance.
- all the generated points are 3D, and in the \[-10 , 10\] cube.

All points follow the equation:

y = a x_0 + b x_1 + c + Eps

This class should allow you to generate any number of points.

Build a class method that allow you to generate training,  and testing datasets:
- the method takes as input the total size of the datast, and the ratio of points used to train (between 0 and 1).

Implement a method that allows you to split the training set into minibatches.
This method will return lists of successive minibatches of data.
Each minibatch will have the same size, and datapoints which are not assigned to the last minibatch will be discarded.

Finally, implement a method that allows to display points, as well as :
- the actual line that was used to generate the points
- an optional additional line


In [148]:
 # We will use the followig command to allow visualization of 3D plots
%matplotlib qt

import numpy as np
import math
from numpy.random import default_rng
rng = default_rng()

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class Line3D:
    
    def __init__( self, line_parameters = (2, 4, -1), noise_variance = 1 ):
        
        self.line_parameters = line_parameters
        self.noise_variance = noise_variance
        
    def generate_points(self, N):

        # Adding the X_0 coordinates full of ones cost us nothing, but will allow us to use vector notation and calculation. 
        X_0 = np.ones(N)
        X_1 = rng.uniform(-10, 10, N)
        X_2 = rng.uniform(-10, 10, N)
        
        epsilon = rng.normal(0, self.noise_variance**(1/2), N)
         
        theta_0, theta_1, theta_2 = self.line_parameters
        
        y = theta_0 + X_1 * theta_1 + X_2 * theta_2 + epsilon
        
        # We create a design matrix, where rows are datapoints and columns are features, or input dimensions
        X = np.vstack( [X_0, X_1, X_2]).transpose()
                
        return X, y
        
    def generate_dataset(self, N, ratio_train = 0.6):
        
        self.X_train , self.y_train = self.generate_points( int(N*ratio_train) )
        self.X_test , self.y_test = self.generate_points( N - int(N*ratio_train) )

    def get_minibatches_train(self, M):
            
        size_split = int(len(self.X_train)/M)
        
        X_minibatch = []
        y_minibatch = []
        
        for i in range(M):
            
            X_mini = self.X_train[ i * size_split : (i+1)*size_split, : ]
            y_mini = self.y_train[ i * size_split : (i+1)*size_split ]
            
            X_minibatch.append(X_mini)
            y_minibatch.append(y_mini)
            
        return X_minibatch, y_minibatch
    
    def plot( self, X, y, additional_line_parameters = None ):
        
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111, projection='3d')
        
        ax.scatter(X[:, 1], X[:, 2], y, marker = '.')
        
        X_1 = np.linspace(-10, 10, 2)
        X_2 = np.linspace(-10, 10, 2)

        X_1, X_2 = np.meshgrid(X_1, X_2)
        
        if additional_line_parameters is not None:
            theta_0, theta_1, theta_2 = additional_line_parameters
        else:
            theta_0, theta_1, theta_2 = self.line_parameters
        
        y = theta_0 + X_1 * theta_1 + X_2 * theta_2
    
        ax.plot_surface(X_1, X_2, y, alpha = 0.4, color='g')
        

data = Line3D( line_parameters=(1,2,3), noise_variance=100 )
X_vis, y_vis = data.generate_points(1000)
data.plot(X_vis, y_vis)

    

### What happens here? Why do we have a plan instead of a line? 

So, this was confusing on purpose. In 2D, a Linear Regression deals with fitting a model to a line.

However, in 3D, the general linear equation y = a * x_1 + b * x_2 + c doesn't correspond to a line, but to a plan.

More generally, the linear equation y = sum( a_i * x_i ) corresponds to a hyperplan of dimension N-1.

So remember, that talking about linear model doesn't always mean that we are dealing with a line.



## Exercise 2

Now, we will solve Linear Regression using the different methods we saw in class.

First of all, we will build an Abstract Base Class that will define how all Linear Regression subclasses behave.
Some methods are common to all subclasses that we will implement.

We will evaluate all our methods using the r2 score.



In [149]:
from abc import ABC, abstractmethod


class LinearRegression(ABC):

    def __init__(self):
        
        self.plan_params = None
        
    def cost(self, X, y):
        
        # Note that another valid formulation of the cost is the Mean Squared Error, where the sum of squared error is normalized by the number of samples.
        
        y_pred = self.predict(X)
        
        calculated_cost = np.sum( (y-y_pred)**2 )
        
        return calculated_cost
        
        
    # This will need to be implemented by any Class inheriting from LinearRegression    
    @abstractmethod
    def fit(self, X, y):
        # each subclass will reimplement this
        pass
    
    def predict(self, X):
        
        theta_0, theta_1, theta_2 = self.plan_params
        
        # We can use vector notation eqivalent to :
        # y_pred = X[:,0] * theta_0 + X[:,1] * theta_1 + X[:,2] * theta_2
        y_pred = X.dot(self.plan_params)
        
        return y_pred
    
    def evaluate(self, X_test, y_test):
        
        y_mean = np.mean(y_test)
        
        y_pred = self.predict(X_test)
        
        # Total Sum of Squares
        SS_tot = np.sum( (y_test - y_mean)**2 )
        
        # Residal Sum of Squares
        SS_res = np.sum( (y_test - y_pred)**2 )
        
        # Coefficient of determination
        r2 = 1 - SS_res/SS_tot
        
        return r2
    
    def plot(self, X, y):
        
        # plot the points with the plane defined by the current parameters.
        
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(X[:, 1], X[:, 2], y, marker = '.')
        
        X_1 = np.linspace(-10, 10, 2)
        X_2 = np.linspace(-10, 10, 2)

        X_1, X_2 = np.meshgrid(X_1, X_2)
        
        theta_0, theta_1, theta_2 = self.plan_params
        
        y = theta_0 + X_1 * theta_1 + X_2 * theta_2
    
        ax.plot_surface(X_1, X_2, y, alpha = 0.4, color='g')

Now that the Base class is defined, we can create different subclasses that implement the appraoches we saw during the Lecture.

In [150]:
from numpy.linalg import inv

class OrdinaryLeastSquare(LinearRegression):
    
    def fit(self, X, y):
        
        self.plan_params = inv(X.transpose().dot(X) ).dot(X.transpose()).dot(y)
        

X_train, y_train = data.generate_points(10000)

ols = OrdinaryLeastSquare()
ols.fit(X_train, y_train)

print(ols.plan_params)

[0.88221872 2.00774937 2.99561708]


We see that that we are not too far off. 

We can check the r2 score

In [151]:
X_test, y_test = data.generate_points(1000)
r2 = ols.evaluate(X_test, y_test)

print(r2)

0.8235496864833776


In [164]:

class GradientDescent(LinearRegression):
    
    def __init__(self, learning_rate):
        
        self.lr = learning_rate
        
        self.plan_params = rng.normal(size = 3)
        
    def mse(self, X, y):
        
        return self.cost(X,y)/len(y)
    
    def fit(self, X_train, y_train, number_epochs ):
        
        
        cost_history = []
        
        for epoch in range(number_epochs):
            
            # we can use vector calculation to compute the value of the gradient
            gradient = -(y_train - self.predict(X_train)).transpose().dot(X_train)/len(X_train)
            
            self.plan_params -= self.lr * gradient  
            
            cost = self.cost(X_train, y_train)
            
            cost_history.append(cost)
            
        return cost_history
    


In [165]:
X_train, y_train = data.generate_points(10000)

gd = GradientDescent(learning_rate=0.01)
cost_hist = gd.fit(X_train, y_train, 100)

plt.plot(cost_hist)
print(gd.plan_params)

[0.68106055 1.99962001 3.02096707]


In [176]:
class StochasticGradientDescent(GradientDescent):
    
    def fit(self, minibatches, number_epochs):
        
        # We can use the fact that we already implemented the update rule!
        
        cost_history_epochs = []
        cost_history_updates = []
                
        for epoch in range(number_epochs):
            
            for X_train_mini, y_train_mini in zip(minibatches[0], minibatches[1]):
                
                cost_history_minibatch = super().fit(X_train_mini, y_train_mini, number_epochs = 1)
                
                # There is only one element in cost_history_minibatch, as we only updated for one epoch
                cost_history_updates.append( cost_history_minibatch[0] )
                
            cost_history_epochs.append(cost_history_updates[-1])
            
        return cost_history_epochs, cost_history_updates

                
data.generate_dataset( 10000, ratio_train = 0.8)
        
minibatches = data.get_minibatches_train( 60 )
sgd = StochasticGradientDescent(learning_rate=0.001)
cost_history_epochs, cost_history_updates = sgd.fit(minibatches, 100)

# plt.plot(cost_history_epochs)
plt.plot(cost_history_updates[:1000])

print(sgd.plan_params)           
        
        

[1.04554411 2.01849998 3.00176249]


We observe a certain periodicity in the Loss, why is that?

it has to do with the way we built the minibatches!
In principle, it is always a good idea to shuffle the minibatch after each epoch.

## Exercise 3

Solve the linear regression problem for different values of line parameters.

Compare the speed of convergence of different approaches depending on the learning rate.

What happens if the learning rate is too high?



## Exercise 4

Use scikit-learn to verify that you obtain the same results.